In [2]:
import pandas as pd
import os
import statsmodels.api as sm
import numpy as np
data_path = "/Users/macbook/Downloads"
data = pd.read_csv(os.path.join(data_path,"test_sample.csv"))

In [3]:
data.head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491
0,-107.498970,-1.049926,5.442844,-0.790073,-0.468225,1.283172,0.264142,-3.322775,-4.711903,1.564933,...,-1.270605,0.793355,-2.459060,1.715212,2.708363,2.073760,0.341741,0.000254,4.552542,-0.926013
1,31.679759,2.087927,0.462747,-2.325459,-0.838933,-2.505421,-2.377476,-3.107347,2.092951,-2.353978,...,-0.225959,0.174485,0.113080,-0.225924,0.664547,1.257062,-0.130490,-1.035967,2.502788,0.288226
2,104.007124,-0.208435,0.533052,3.299618,2.432931,-1.883478,-0.491675,-0.379904,2.703601,1.196047,...,-3.744385,0.891021,0.442570,-2.170280,-2.032674,3.115616,-1.109971,1.627791,1.431372,0.601611
3,-16.596949,0.890755,-2.410997,0.042064,-2.094164,-1.795909,-1.245525,-2.148149,1.648515,0.654845,...,-3.125350,-3.281282,1.812526,-2.359601,-0.283134,1.628851,-1.058098,1.117114,-2.189131,1.033255
4,-45.289213,-2.277988,-0.974078,0.696096,0.317506,-1.091769,0.555166,-2.347522,4.292175,1.556668,...,1.837415,-2.779156,-0.991246,0.605518,2.893190,2.003156,-4.601483,-1.974966,2.009548,1.935258


In [4]:
Y = data.Y
X = data.drop("Y", axis = 1)
X.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491
0,-1.049926,5.442844,-0.790073,-0.468225,1.283172,0.264142,-3.322775,-4.711903,1.564933,-2.265081,...,-1.270605,0.793355,-2.459060,1.715212,2.708363,2.073760,0.341741,0.000254,4.552542,-0.926013
1,2.087927,0.462747,-2.325459,-0.838933,-2.505421,-2.377476,-3.107347,2.092951,-2.353978,-2.515340,...,-0.225959,0.174485,0.113080,-0.225924,0.664547,1.257062,-0.130490,-1.035967,2.502788,0.288226
2,-0.208435,0.533052,3.299618,2.432931,-1.883478,-0.491675,-0.379904,2.703601,1.196047,-0.178445,...,-3.744385,0.891021,0.442570,-2.170280,-2.032674,3.115616,-1.109971,1.627791,1.431372,0.601611
3,0.890755,-2.410997,0.042064,-2.094164,-1.795909,-1.245525,-2.148149,1.648515,0.654845,-0.416846,...,-3.125350,-3.281282,1.812526,-2.359601,-0.283134,1.628851,-1.058098,1.117114,-2.189131,1.033255
4,-2.277988,-0.974078,0.696096,0.317506,-1.091769,0.555166,-2.347522,4.292175,1.556668,-3.024288,...,1.837415,-2.779156,-0.991246,0.605518,2.893190,2.003156,-4.601483,-1.974966,2.009548,1.935258


In [5]:
def rel_imp_me(X, y): #штука, считающая вклады в R^2 разными способами
    names = X.columns
    ser = pd.Series(index = names)
    lm0 = sm.OLS(y, sm.add_constant(X)).fit()
    for c in names:
        lm = sm.OLS(y, sm.add_constant(X[names.drop(c)])).fit()
        ser[c] = lm0.rsquared - lm.rsquared 
    res = pd.DataFrame(columns =['last', 'first', 'betasq', 'pratt'], index=names)
    res['last'] = ser
    corr = X.apply(lambda x: np.corrcoef(y,x)[0,1], axis=0)
    res['first'] = corr**2
    sx = X.std()
    res['betasq'] = (lm0.params[names] * sx / np.std(y))**2
    res['pratt'] = (lm0.params[names] * sx / np.std(y)) * corr
    return res

In [6]:
import statsmodels.api as sm
fits = [sm.OLS(Y, X.iloc[:,:j]).fit() for j in range(2, 492)]
rSquared = [fit.rsquared for fit in fits] 

In [18]:
n_orig = 0
for i in range(len(rSquared)):
    if (rSquared[i] > 0.9):
        n_orig = i + 2
        break
print(n_orig)

335


In [7]:
from sklearn.decomposition import PCA
nFactors = 491
xPCA = PCA(n_components=nFactors)
xPCA.fit(X)

PCA(n_components=491)

In [9]:
factorScores = pd.DataFrame(np.dot(X, xPCA.components_.T), columns =["PC%i"%(j+1) for j in range(nFactors)])
factorScores.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC482,PC483,PC484,PC485,PC486,PC487,PC488,PC489,PC490,PC491
0,3.857767,-6.090354,-4.120492,-6.372533,-2.903906,-1.448779,0.700898,3.508689,-0.917566,1.378988,...,-0.159357,0.063217,0.033263,0.158692,0.147235,-0.003213,0.004375,-0.060743,0.047344,0.187070
1,-1.822760,-1.797609,-2.938746,5.308121,-2.585913,-2.230793,-1.516077,1.263123,-1.577026,-4.418641,...,-0.213288,0.186224,0.059475,0.265986,0.235854,0.104916,-0.006259,-0.051871,0.038899,0.188885
2,-2.017898,-6.650528,1.541173,-0.179140,-3.178328,1.236105,2.401075,4.220357,1.703371,0.128007,...,-0.125967,0.043608,0.061520,0.150295,0.089292,0.034726,-0.048014,-0.045703,0.059864,0.207219
3,3.178331,-5.832328,4.036040,0.122306,-3.598300,0.481620,7.186330,3.269082,-2.093393,-0.416380,...,-0.065989,0.105187,0.145645,0.126325,0.197663,-0.054364,-0.019019,-0.035901,0.110696,0.168643
4,-2.673100,-1.914584,2.871642,1.521642,-1.275889,0.278117,-1.724477,4.072124,2.196421,4.492407,...,-0.023363,0.150206,0.084866,0.367047,0.157015,0.113518,-0.001875,-0.023871,0.083242,0.228955


In [10]:
import warnings
warnings.filterwarnings('ignore')
metricsPCA = rel_imp_me(factorScores, Y)
metrics_first_rankPCA = metricsPCA["first"].rank(ascending=False, method='first')
metrics_first_rankPCA.head()

/var/folders/x6/b_ylqyd127s0ckfw_r19b2880000gn/T/ipykernel_49634/1145815622.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ser = pd.Series(index = names)


PC1      4.0
PC2    181.0
PC3    238.0
PC4    147.0
PC5    107.0
Name: first, dtype: float64

In [11]:
PC_sorted = pd.DataFrame({"Predictors" : metrics_first_rankPCA.index,
  "Rank" : metrics_first_rankPCA.values}).sort_values(by="Rank")
PC_sorted.head()

,Predictors,Rank
13,PC14,1.0
26,PC27,2.0
22,PC23,3.0
0,PC1,4.0
48,PC49,5.0


In [12]:
orderedPC = pd.DataFrame(factorScores, columns=PC_sorted["Predictors"])
orderedPC.shape

(500, 491)

In [13]:
fitsPCA = [sm.OLS(Y, orderedPC.iloc[:,:j]).fit() for j in range(2, 492)]
rSquaredPCA = [fitPCA.rsquared for fitPCA in fitsPCA] 

In [14]:
n_PCA = 0
rs = 0
for j in range(len(rSquaredPCA)):
    if (rSquaredPCA[j] > 0.9):
        n_PCA = j + 2
        rs = rSquaredPCA[j]
        break
n_PCA

138

In [19]:
print("Model dimensionality reduction:", n_orig - n_PCA, "\nDetermination coefficient:", rs)

Model dimensionality reduction: 197 
Determination coefficient: 0.9000572073467212


In [16]:
335 - 138

197